In [ ]:
from os import walk
import tarfile
#解压csv文件
tar_file = []
mypath = "D:\\fyp\\predataset"#对应文件的路径
for (dirpath, dirnames, filenames) in walk(mypath):
    tar_file.extend(filenames)
    break

for word in tar_file:
    filename = mypath + "\\" + word
    tf = tarfile.open(filename)
    tf.extractall('D:/fyp/dataset')

In [1]:
import pandas as pd
data_12 = pd.read_csv('D:/fyp/dataset/2020-12.csv')

In [36]:
a = "我爱学习study_palu# 学✅血常习hi是谁111"
b = re.sub(r'[^\w+#]'," ",a)
b

'我爱学习study_palu# 学 血常习hi是谁111'

In [2]:
#预处理
import jieba
import re
import emoji

#正则清洗
def clean_re(text):
    new_text =  re.sub(r'[^\w+#]'," ",text)
    new_text = re.sub(r"[A-Za-z0-9_.!+-=——,$%^，。？、~@￥%……&*《》<>「」{}【】()/]", " ",new_text)
    new_text = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", " ", new_text)  # 去除正文中的@和回复/转发中的用户名
    new_text = emoji.demojize(new_text)
    new_text = re.sub(r"\[\S+\]", "",new_text)      # 去除表情符号
    #new_text = re.sub(r"#\S+#", "", new_text)      # 保留话题内容
    URL_REGEX = re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',re.IGNORECASE)
    new_text = re.sub(URL_REGEX, "",new_text)       # 去除网址
    new_text = new_text.replace("转发微博", "")       # 去除无意义的词语
    new_text = new_text.replace("转存", "")       # 去除无意义的词语
    new_text = new_text.replace("超话", "")       # 去除无意义的词语
    new_text = re.sub(r"\s+", " ",new_text) # 合并正文中过多的空格
    new_text = new_text.strip()
    return new_text


#读入停用词
def stopwords_list(path):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

#分词+去除停用词
def cut_move(text):
    wlist = jieba.cut(text)
    words = [w for w in wlist if w not in stopwords]
    return " ".join(words)


filepath = 'D:/fyp/stopwords.txt'
stopwords = stopwords_list(filepath) #读入停用词



In [3]:
# 正则清洗
data_12['clean_review'] = data_12.content.apply(clean_re)

In [4]:
data_12['corpus'] = data_12.clean_review.apply(cut_move)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86199\AppData\Local\Temp\jieba.cache
Loading model cost 1.404 seconds.
Prefix dict has been built successfully.


In [5]:
data_12['corpus'].to_csv("D:/fyp/corpus_12.csv",sep=',',index=False)

In [9]:
import pickle
# 保存每条分词结果的list, 每个元素是一个list
comment_list1 = data['corpus'].tolist()
comment_list = [str.split(" ") for str in comment_list1]
pickle.dump(comment_list, open('D:/fyp/content.pkl', 'wb'))# pickle.dump () 封装是一个将Python数据对象转化为字节流的过程

In [10]:
'''
词云
'''
import wordcloud
content = ''
# 存储每个词语出现的次数
word_dict = {}
# print(len(content_comment))
for i in comment_list:
    try:
        content += (' ' + ' '.join(i))
        for word in i:
            # 统计词频  一个字忽略
            if len(word) < 2:
                continue
            if word_dict.get(word):
                word_dict[word] += 1
            else:
                word_dict[word] = 1
    except:
        continue
# 出现次数最多的50个词语
top50words=list(word_dict.items())
top50words.sort(key=lambda x:x[1],reverse=True)
words50 = ""

for i in range(50):
    words50 += top50words[i][0] + ' '
    # print(top50words[i][0], top50words[i][1])

w = wordcloud.WordCloud(width=1000, font_path='./msyh.ttc', background_color='white', height=700, stopwords={"知道", "觉得", "中国", "国家", "评论", "大家", "所有", "必须", "之前", "需要", "哈哈哈", "哈哈哈哈", "真的", "这种", "没有", "不会", "起来", "一点", "已经", "啊啊啊", "可能", "今天", "现在", "很多", "出来", "关注", "即可", "看到", "希望"})
w.generate(words50.strip())
w.to_file('D:/fyp/wc_50.png')

# 2. LDA主题模型分析

In [6]:
import pandas as pd

data = pd.read_csv("D:/fyp/corpus/corpus_0.csv")

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [8]:
# 输出每个主题的关键词
def print_top_words(model, feature_name, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d" % topic_idx)
        topic_w = " ".join([feature_name[i] for i in topic.argsort()[:-n_top_words - 1: -1]])
        tword.append(topic_w)
        print(topic_w)
    return tword


In [10]:
# 提取1000个特征词语
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df=0.5,
                                min_df=10)
tf = tf_vectorizer.fit_transform(data['corpus'].astype('U'))

In [11]:
n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50,
                                doc_topic_prior=0.1,
                                topic_word_prior=0.01,
                               random_state=0)
lda.fit(tf)

LatentDirichletAllocation(doc_topic_prior=0.1, learning_offset=50, max_iter=50,
                          random_state=0, topic_word_prior=0.01)

### 2.1输出每个主题对应词语

In [15]:
n_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic #0
肺炎 武汉 原因 不明 发现 武汉市 患者 第一 检测 工作
Topic #1
亚洲 严重 唯一 公司 终于 成就 奖项 韩国 世界 取得
Topic #2
法医 女性 捐者 接受 骨髓移植 男性 角度看 同事 消失 哥哥
Topic #3
中方 香港 美国 民主 美方 制裁 中国 组织 国际 非政府
Topic #4
病例 肺炎 病情 病毒 相关 病毒性 导弹 进行 目前 海鲜城
Topic #5
医生 检查 治疗 感染 发现 出现 可能 疾病 症状 医院
Topic #6
中国 没有 美国 日本 公司 大家 不是 问题 非常 很多
Topic #7
孩子 流感 发烧 家长 使用 首次 服用 患者 宝宝 儿童
Topic #8
艾滋病 兰州 mark 阳性 感染 exclamation 皮肤 红枣 报告 调理
Topic #9
医生 时光 患者 医院 医保 礼物 学生 地铁 周年 影响


In [16]:
import numpy as np
topics=lda.transform(tf)
topic = []
for t in topics:
    topic.append("Topic #"+str(list(t).index(np.max(t))))
data['概率最大的主题序号']=topic
data['每个主题对应概率']=list(topics)
data.to_excel("D:/fyp/data_topic.xlsx",index=False)

In [17]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pic = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.display(pic)
pyLDAvis.save_html(pic, 'lda_pass'+str(n_topics)+'.html')
pyLDAvis.display(pic)


#浅蓝色的表示这个词在整个文档中出现的频率（权重），深红色的表示这个词在这个主题中所占的权重。右上角可以调节一个参数λ
#如果λ接近1，那么在该主题下更频繁出现的词，跟主题更相关；
#如果λ越接近0，那么该主题下更特殊、更独有（exclusive）的词，跟主题更相关（有点TF-IDF的意思了）
#气泡的大小及编号即表示主题出现的频率

D:\Anaconda\anaconda\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
D:\Anaconda\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
D:\Anaconda\anaconda\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [17]:
pip install --upgrade smart_open

Note: you may need to restart the kernel to use updated packages.


     --------------------------------------- 56.8/56.8 kB 96.1 kB/s eta 0:00:00
  Attempting uninstall: smart_open
    Found existing installation: smart-open 6.1.0
    Uninstalling smart-open-6.1.0:
      Successfully uninstalled smart-open-6.1.0


In [1]:
import pandas as pd

data_10 = pd.read_csv('D:/fyp/corpus/corpus_10.csv')

In [2]:
from gensim.models import Word2Vec

import gensim
import jieba

# 定义分词函数
def cut_words(sentence):
    return [word for word in jieba.cut(sentence) if word.strip()]



In [4]:
# 预处理语料库
sentences = [cut_words(str(text)) for text in data_10['corpus']]


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86199\AppData\Local\Temp\jieba.cache
Loading model cost 0.565 seconds.
Prefix dict has been built successfully.


In [ ]:
import numpy as np

s = np.array(sentence)
np.save('D:/fyp/corpus/demo10.npy',s)


In [ ]:
#词向量保存
#model.wv.save_word2vec_format('D:/fyp/2020-02.vector', binary=False)
    
#模型保存

#model.wv.save_word2vec_format('D:/fyp/2019-12.bin')

In [ ]:
model = Word2Vec(sentences,vector_size=200,alpha=0.025, window=5, min_count=10,
                 min_alpha=0.0025,sg=1, hs=0, negative=5, epochs=50)


model.save('D:/fyp/word2vec_10.model')

In [42]:
model = Word2Vec.load('D:/fyp/corpus_w2v_weibo.model')